# Carga y preprocesamiento de los datos

In [ ]:
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 6.0 MB/s eta 0:00:00


In [ ]:
#Librerias
import os
import re
from datetime import datetime
from unidecode import unidecode
from langchain.document_loaders import PyMuPDFLoader
from langchain.schema import Document
import nltk
import pinecone
from tqdm import tqdm
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Pinecone as PineconeStore
from pinecone import Pinecone, ServerlessSpec
import requests
from dotenv import load_dotenv

load_dotenv()

ModuleNotFoundError: No module named 'unidecode'

In [ ]:
!pip install langchain==0.1.16 # Install a compatible version of langchain
!pip install langchain-core==0.1.48 # Specify the required version
!pip install langchain-text-splitters==0.0.1 # Specify the required version
!pip install pymupdf
!pip install langchain-community==0.0.33 # Install a compatible version of langchain-community
!pip install pinecone
!pip install unidecode
import nltk
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.1/303.1 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: langsmith
    Found existing ins

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.53
    Uninstalling langchain-core-0.1.53:
      Successfully uninstalled langchain-core-0.1.53
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.0.38 requires langchain-core<0.2.0,>=0.1.52, but you have langchain-core 0.1.48 which is incompatible.
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.0.2
    Uninstalling langchain-text-splitters-0.0.2:
      Successfully uninstalled langchain-text-splitters-0.0.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 25.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-community
    Found

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 5.8 MB/s eta 0:00:00


True

In [ ]:
# Codigo para tener acceso a archivos de Drive
from google.colab import drive
drive.mount('/content/drive')
# Configurar direcciones de dataset
dir_crudo = "dataset/dataset_crudo"
dir_aumentado = "/content/drive/MyDrive/dataset/dataset_aumentado"
RUTA_APUNTES = "/content/drive/MyDrive/Apuntadores"
RUTA_SALIDA = "/content/drive/MyDrive/outputs"
os.makedirs(RUTA_SALIDA, exist_ok=True)
INDEX_NAME = "apuntes-ia"
DIMENSION = 1536
REGION = "us-east-1"
ruta_parrafos = "/content/drive/MyDrive/outputs/chunks_parrafo.txt"
ruta_frases   = "/content/drive/MyDrive/outputs/chunks_frase.txt"

Mounted at /content/drive


In [ ]:
def limpia_texto(texto: str) -> str:
    texto = re.sub(r"\$[^$]+\$", " [fórmula] ", texto)        # Fórmulas inline
    texto = re.sub(r"\\\[.*?\\\]", " [fórmula] ", texto)      # Fórmulas multilínea
    texto = re.sub(r"\\[a-zA-Z]+\{[^}]*\}", " ", texto)       # Comandos tipo \text{...}
    texto = re.sub(r"\\[a-zA-Z]+", " ", texto)                # Comandos tipo \alpha
    texto = re.sub(r"\s+", " ", texto)                        # Espacios extra
    texto = unidecode(texto)
    return texto.strip()

In [ ]:
def cargar_documentos(ruta=RUTA_APUNTES):
    documentos = []
    for archivo in os.listdir(ruta):
        if archivo.lower().endswith(".pdf"):
            path = os.path.join(ruta, archivo)
            loader = PyMuPDFLoader(path)
            pdf_docs = loader.load()

            for doc in pdf_docs:
                texto_limpio = limpia_texto(doc.page_content)
                metadata = {
                    "archivo": archivo,
                    "pagina": doc.metadata.get("page", 0),
                    "fecha_procesado": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                }
                documentos.append(Document(page_content=texto_limpio, metadata=metadata))
            print(f" Procesado: {archivo} ({len(pdf_docs)} páginas)")
    return documentos

In [ ]:
def extraer_autor(texto: str) -> str:
    lineas = [l.strip() for l in texto.split("\n") if l.strip()]
    if len(lineas) >= 2:
        return lineas[1]
    return "Desconocido"

In [ ]:
def segmentar_por_parrafo(documentos):
    chunks = []
    for doc in documentos:
        parrafos = re.split(r'\n\s*\n|(?<=\.)\s{2,}', doc.page_content)
        for p in parrafos:
            p = p.strip()
            if len(p) > 50:  # Evita fragmentos muy cortos
                chunks.append(Document(page_content=p, metadata=doc.metadata))
    print(f" Segmentación por párrafo: {len(chunks)} chunks generados.")
    return chunks

In [ ]:
def segmentar_por_frase(documentos):
    chunks = []
    for doc in documentos:
        frases = nltk.sent_tokenize(doc.page_content)
        for f in frases:
            f = f.strip()
            if len(f) > 30:  # Evita fragmentos demasiado cortos
                chunks.append(Document(page_content=f, metadata=doc.metadata))
    print(f" Segmentación por frase: {len(chunks)} chunks generados.")
    return chunks

In [ ]:
def guardar_chunks(chunks, nombre):
    path = os.path.join(RUTA_SALIDA, f"chunks_{nombre}.txt")
    with open(path, "w", encoding="utf-8") as f:
        for i, chunk in enumerate(chunks):
            f.write(f"[CHUNK {i} - {chunk.metadata}]\n{chunk.page_content}\n\n{'='*80}\n\n")
    print(f" Guardados {len(chunks)} chunks en {path}")

In [ ]:
# 1. Cargar y limpiar PDFs
docs = cargar_documentos(RUTA_APUNTES)

# 2. Generar segmentaciones
chunks_parrafo = segmentar_por_parrafo(docs)
chunks_frase = segmentar_por_frase(docs)

# 3. Guardar en disco
guardar_chunks(chunks_parrafo, "parrafo")
guardar_chunks(chunks_frase, "frase")

 Procesado: 3_Semana_AI_20250819_2.pdf (3 páginas)
 Procesado: 3_Semana_AI_20250821_1.pdf (3 páginas)
 Procesado: 8_Semana_AI_20250925_2.pdf (4 páginas)
 Procesado: 1_SEMANA_AI_20250807_1.pdf (3 páginas)
 Procesado: 5_Semana_AI_20250902_2.pdf (2 páginas)
 Procesado: 4_SEMANA_AI_20250826_2.pdf (4 páginas)
 Procesado: 4_SEMANA_AI_20250826_1.pdf (3 páginas)
 Procesado: 4_Semana_AI_20250828_2.pdf (3 páginas)
 Procesado: 12_Semana_AI_20251021_2.pdf (4 páginas)
 Procesado: 3_Semana_AI_20250819_1.pdf (4 páginas)
 Procesado: 12_SEMANA_AI_20251021_1.pdf (3 páginas)
 Procesado: 6_Semana_AI_20250911_1.pdf (4 páginas)
 Procesado: 5_Semana_AI_20250904_1.pdf (5 páginas)
 Procesado: 8_Semana_AI_20250923_1.pdf (3 páginas)
 Procesado: 9_SEMANA_AI_20251002_1.pdf (9 páginas)
 Procesado: 12_SEMANA_AL_20251023_2.pdf (3 páginas)
 Procesado: 6_Semana_AI_20250911_2.pdf (4 páginas)
 Procesado: 10_SEMANA_AI_20251007_1-222887296.pdf (2 páginas)
 Procesado: 2_Semana_AI_20250814_2.pdf (5 páginas)
 Procesado: 7_Sem

Carga a base de datos vectorial

In [ ]:
def crear_indice():
  # Crear índice si no existe
  if INDEX_NAME not in [i.name for i in pc.list_indexes()]:
      pc.create_index(
          name=INDEX_NAME,
          dimension=DIMENSION,
          metric="cosine",
          spec=ServerlessSpec(cloud="aws", region=REGION)
      )
      print(f"Índice '{INDEX_NAME}' creado en Pinecone.")
  else:
      print(f"Índice '{INDEX_NAME}' ya existe.")

In [ ]:
def cargar_chunks_desde_txt(path):
    chunks = []
    with open(path, 'r', encoding='utf-8') as f:
        contenido = f.read()

    bloques = re.split(r"=+\n+", contenido)
    for bloque in bloques:
        bloque = bloque.strip()
        if not bloque:
            continue
        # Extraer texto y metadatos (opcional)
        partes = bloque.split("]", 1)
        if len(partes) == 2:
            meta_text = partes[0].replace("[CHUNK", "").strip()
            texto = partes[1].strip()
        else:
            texto = bloque
            meta_text = "{}"

        doc = Document(page_content=texto, metadata={"origen": path})
        chunks.append(doc)
    print(f"Cargados {len(chunks)} chunks desde {path}")
    return chunks

In [ ]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index = pc.Index("apuntes-ia")
embeddings = OpenAIEmbeddings(model="text-embedding-3-small",openai_api_key=os.getenv("OPENAI_API_KEY"))

def subir_a_pinecone(chunks, namespace):
    batch_size = 100
    for i in tqdm(range(0, len(chunks), batch_size)):
        batch = chunks[i:i+batch_size]
        texts = [c.page_content for c in batch]
        metas = [c.metadata for c in batch]
        vecs = embeddings.embed_documents(texts)

        to_upsert = []
        for j in range(len(vecs)):
            metadata = metas[j].copy()
            metadata['text'] = texts[j]
            to_upsert.append((f"{namespace}-{i+j}", vecs[j], metadata))

        index.upsert(vectors=to_upsert, namespace=namespace)

    print(f" {len(chunks)} vectores subidos al namespace '{namespace}'")

subir_a_pinecone(chunks_parrafo, "parrafos")
subir_a_pinecone(chunks_frase, "frases")

100%|██████████| 2/2 [00:07<00:00,  3.82s/it]


 165 vectores subidos al namespace 'parrafos'


100%|██████████| 35/35 [01:12<00:00,  2.07s/it]

 3423 vectores subidos al namespace 'frases'


It seems that the Pinecone API key was not found in the Colab secrets manager. You need to add your Pinecone API key to the secrets manager under the "🔑" in the left panel with the name `PINECONE_API_KEY`.